<a href="https://colab.research.google.com/github/rafaelinfopiaui/analise-sentimentos-cpweekend/blob/main/notebooks/Desenvolvimento_Analise_Sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Análise de Sentimentos para a Campus Party Weekend Piauí 2025

**Objetivo:** Desenvolver um protótipo de Machine Learning capaz de classificar o sentimento de textos em português (positivo, negativo, neutro) a partir de dados coletados de redes sociais.

**Projeto de Extensão:** Engenharia de Computação com IA - UNI-CET

**Equipe de Desenvolvedores:**
* Rafael Oliveira
* Ailton Medeiros
* Lais Eulálio
* Antônio Wilker
* Isaac Aragão
* Paula Iranda

**Docente Orientador:** Prof. Dr. Artur Felipe da Silva Veloso

**Etapas deste Notebook:**
1.  **Configuração do Ambiente:** Conexão com o Google Drive e importação de bibliotecas.
2.  **Carga e Análise dos Dados:** Carregar o dataset e fazer uma análise exploratória inicial.
3.  **Pré-Processamento:** Limpeza e preparação dos textos para o modelo.
4.  **Treinamento do Modelo:** Construção, treino e avaliação de um modelo de classificação.
5.  **Salvamento do Modelo:** Exportar o modelo treinado para ser usado na aplicação com Streamlit.

## 1. Configuração do Ambiente

Nesta primeira etapa, a equipe irá conectar o Colab ao Google Drive para acessar os dados, importar todas as bibliotecas que serão utilizadas no projeto e definir os caminhos dos arquivos para manter o código organizado e acessível a todos.

In [18]:
## 1. Configuração do Ambiente

# 1.1 - INSTALAÇÃO E UPGRADE DE PACOTES (CRUCIAL PARA CORREÇÃO)
!pip install pandas scikit-learn streamlit nltk joblib
!pip install numpy --upgrade
!pip install scikit-learn --upgrade
!pip install joblib --upgrade
print("Pacotes de ML instalados/atualizados com sucesso!")

# 1.2 - Conectar ao Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 1.3 - LIMPEZA DE CACHE DO AMBIENTE (FINAL)
# Movemos para o início para que pacotes sejam carregados corretamente
%reset -f
print("\nCache de pacotes limpo. Recarregando ambiente...")

# 1.4 - REDEFINIR Variáveis e Mudar diretório (APÓS O RESET)
# A variável PASTA_PROJETO precisa ser redefinida aqui
PASTA_PROJETO = '/content/drive/MyDrive/Projeto_CampusParty_Sentimentos'
%cd {PASTA_PROJETO}

# 1.5 - Importar bibliotecas essenciais (TUDO SERÁ IMPORTADO A PARTIR DAQUI)
import pandas as pd
import numpy as np
import re
import nltk
import joblib

# 1.6 - Importar módulos do Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1.7 - Definir Caminhos (USANDO A PASTA_PROJETO CORRIGIDA)
CAMINHO_DADOS_BRUTOS = f'{PASTA_PROJETO}/data/dataset_bruto.csv'
CAMINHO_DADOS_LIMPOS = f'{PASTA_PROJETO}/data/dados_limpos.csv'
CAMINHO_MODELO = f'{PASTA_PROJETO}/saved_models/modelo_sentimento.joblib'

print("Ambiente configurado com sucesso!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Ambiente configurado com sucesso!


## 2. Carga e Análise Exploratória dos Dados (EDA)

Nesta etapa, a equipe carrega o dataset e realiza uma verificação rápida para entender sua estrutura, a quantidade de dados e a presença de valores nulos.

**Nota sobre o Fluxo de Trabalho:** Para garantir a estabilidade e contornar um problema persistente de sincronização com o Google Drive, o carregamento dos dados será feito através do **upload direto** para a sessão do Colab a cada vez que o notebook for iniciado. O código a seguir já está preparado para ler o arquivo deste ambiente local temporário, utilizando os parâmetros de leitura que se provaram eficazes durante os testes.

In [19]:
## 2. Carga e Análise Exploratória dos Dados (EDA)

# Define o nome do arquivo que esperamos que o usuário envie
nome_do_arquivo = 'brasil_subreddit_comments.csv'

# Verifica se o arquivo já não foi enviado para a sessão atual
if not os.path.exists(nome_do_arquivo):
  print(f"Por favor, faça o upload do arquivo '{nome_do_arquivo}'.")
  print("Aguardando o upload...")

  # Gera o botão de upload
  uploaded = files.upload()

  # Verifica se o upload foi bem sucedido
  if nome_do_arquivo in uploaded:
    print(f"\n✅ Upload de '{nome_do_arquivo}' concluído com sucesso!")
else:
  print(f"O arquivo '{nome_do_arquivo}' já existe na sessão. Pulando o upload.")

O arquivo 'brasil_subreddit_comments.csv' já existe na sessão. Pulando o upload.


In [20]:
import csv

# O nome do arquivo já foi definido e enviado na célula anterior
nome_do_arquivo = 'brasil_subreddit_comments.csv'

try:
    # Leitura robusta do arquivo CSV com parâmetros específicos
    df = pd.read_csv(
        nome_do_arquivo,
        engine='python',
        quoting=csv.QUOTE_NONE,
        on_bad_lines='skip',
        encoding='latin-1'
    )

    print("✅ SUCESSO! Dataset carregado do armazenamento local da sessão.")

    print("\nPrimeiras 5 linhas do dataset:")
    display(df.head())

    print("\nInformações do DataFrame:")
    df.info(show_counts=True)

except Exception as e:
    print(f"Ocorreu um erro ao ler o arquivo: {e}")

Dataset carregado com sucesso! Total de linhas: 872153

Primeiras 5 linhas do dataset (após mapeamento):


,body,score,texto,sentimento
0,Limpa os cacos com um pano úmido e borrifa àgu...,1,Limpa os cacos com um pano úmido e borrifa àgu...,0
1,Eu autorizo Xandão,1,Eu autorizo Xandão,0
2,Tem tanta coisa esquisita acontecendo nessa ci...,1,Tem tanta coisa esquisita acontecendo nessa ci...,0
3,"Não é caso de educação, caráter você nasce co...",1,"Não é caso de educação, caráter você nasce co...",0
4,Flan é pudim,1,Flan é pudim,0



Informações do DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872153 entries, 0 to 872152
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   author         872153 non-null  object
 1   body           872151 non-null  object
 2   created_utc    872153 non-null  object
 3   distinguished  8641 non-null    object
 4   edited         872153 non-null  object
 5   comment_id     872153 non-null  object
 6   is_submitter   872153 non-null  bool  
 7   parent_id      872153 non-null  object
 8   score          872153 non-null  int64 
 9   stickied       872153 non-null  bool  
 10  thread_id      872153 non-null  object
 11  version        872153 non-null  int64 
 12  texto          872151 non-null  object
 13  sentimento     872153 non-null  int64 
dtypes: bool(2), int64(3), object(9)
memory usage: 81.5+ MB

Distribuição inicial dos sentimentos (0=Negativo/Neutro, 1=Positivo):
sentimento
1    0.641596
0 

## 3. Limpeza Inicial do DataFrame

Com os dados brutos carregados no DataFrame `df`, o primeiro passo é uma "faxina" estrutural para tornar o dataset mais limpo e fácil de trabalhar. As seguintes ações serão tomadas:

1.  **Limpeza dos Nomes das Colunas:** As aspas extras (`"`) presentes nos nomes das colunas serão removidas.
2.  **Remoção de Linhas Vazias:** As linhas que não contêm comentários (valores nulos na coluna `body`) serão eliminadas, pois são inúteis para a nossa análise de sentimentos.
3.  **Seleção de Colunas Relevantes:** Um novo DataFrame, `df_final`, será criado contendo apenas as colunas mais importantes para o nosso estudo (`author`, `body`, `score`, `created_utc`).

In [21]:
print("--- Iniciando Limpeza Inicial do DataFrame ---")

# 1. Limpar os nomes das colunas (remover aspas)
# O .str.replace('"', '') encontra e substitui as aspas por nada.
df.columns = df.columns.str.replace('"', '')
print("✅ Nomes das colunas limpos!")

# 2. Remover linhas onde o comentário ('body') está vazio
# O .dropna() remove as linhas com valores nulos na coluna especificada.
df_limpo = df.dropna(subset=['body'])
print(f"Dataset original tinha {len(df):,} linhas.")
print(f"Dataset limpo (sem comentários vazios) tem {len(df_limpo):,} linhas.")

# 3. Selecionar apenas as colunas que nos interessam para o projeto
# .copy() evita avisos futuros do pandas
df_final = df_limpo[['author', 'body', 'score', 'created_utc']].copy()

print("\n--- Limpeza Concluída! ---")
print("\nAmostra do dataset final (df_final):")
display(df_final.head())

print("\nNovas informações do DataFrame (df_final):")
df_final.info(show_counts=True)

--- Iniciando Limpeza Inicial do DataFrame ---
✅ Nomes das colunas limpos!
Dataset original tinha 1,177,986 linhas.
Dataset limpo (sem comentários vazios) tem 911,662 linhas.

--- Limpeza Concluída! ---

Amostra do dataset final (df_final):


,author,body,score,created_utc
"""fe52f47f-fccd-593a-934b-2912d8cdc9aa""","""Limpa os cacos com um pano Ãºmido e borrifa Ã...",COM CUIDADO pra nÃ£o queimar o forro do colch...,1,"""2025-09-30 17:05:13"""
"""05c74961-e915-5b52-a517-30716352cb8e""","""Eu autorizo XandÃ£oÂ ""","""2025-09-30 17:04:31""",False,""""""
"""126f9c7d-5f8c-513f-9ec9-2ea7f70cde5a""","""Tem tanta coisa esquisita acontecendo nessa c...","""2025-09-30 17:04:17""",False,""""""
"""43e862df-c416-5ba9-a206-ba225543536e""","""NÃ£o Ã© caso de educaÃ§Ã£o",carÃ¡ter vocÃª nasce com ele para o bem ou p...,1,"""2025-09-30 17:03:37"""
"""7ef15e91-8b8c-50ff-a64d-013af21eebfb""","""Flan Ã© pudim""","""2025-09-30 17:03:18""",False,""""""



Novas informações do DataFrame (df_final):
<class 'pandas.core.frame.DataFrame'>
Index: 911662 entries, "fe52f47f-fccd-593a-934b-2912d8cdc9aa" to De ante mÃ£o jÃ¡ te falo que se vocÃª estÃ¡ lendo bons livros
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   author       911656 non-null  object
 1   body         911662 non-null  object
 2   score        729114 non-null  object
 3   created_utc  824458 non-null  object
dtypes: object(4)
memory usage: 34.8+ MB


## 4. Pré-processamento Profundo do Texto

Com o DataFrame estruturalmente limpo, o próximo passo é processar o conteúdo da coluna `body`. Nesta etapa, vamos aplicar uma função de limpeza em cada comentário para:

- Padronizar o texto (converter para letras minúsculas).
- Remover "ruídos" como URLs, menções de usuários e caracteres especiais.
- Remover palavras comuns que não carregam sentimento (stopwords como 'o', 'a', 'de', 'que').

O resultado será salvo em uma nova coluna, `body_limpo`, que servirá de base para o nosso modelo de Machine Learning.

In [22]:
import nltk
import re

# Definindo a função que fará a limpeza profunda em cada texto
def limpar_texto(texto):
    # Converter para minúsculas
    texto = texto.lower()
    # Remover URLs
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto)
    # Remover menções (@) e hashtags (#) - comum em redes sociais
    texto = re.sub(r'@\w+|#\w+', '', texto)
    # Remover caracteres não-alfabéticos (mantém apenas letras e espaços)
    texto = re.sub(r'[^a-z\s]', '', texto)
    # Tokenização (dividir em palavras)
    tokens = nltk.word_tokenize(texto)
    # Remover stopwords
    stopwords_pt = nltk.corpus.stopwords.words('portuguese')
    tokens_limpos = [palavra for palavra in tokens if palavra not in stopwords_pt]
    # Juntar os tokens de volta em uma string
    return ' '.join(tokens_limpos)

print("--- Iniciando Pré-processamento Profundo do Texto ---")
print("Isso pode levar um momento, pois estamos processando mais de 30,000 comentários...")

# O .apply() executa a função 'limpar_texto' em cada linha da coluna 'body'
df_final['body_limpo'] = df_final['body'].apply(limpar_texto)

print("\n✅ Pré-processamento do texto concluído!")

# Vamos ver o resultado da limpeza
print("\nComparação do texto original vs. texto pré-processado:")
display(df_final[['body', 'body_limpo']].head())

[Passo 1/2] Aplicando limpeza profunda do texto...

✅ Pré-processamento do texto concluído!

Comparação do texto original vs. texto pré-processado:


,body,body_limpo
"""fe52f47f-fccd-593a-934b-2912d8cdc9aa""",COM CUIDADO pra nÃ£o queimar o forro do colch...,cuidado pra queimar forro colcho
"""05c74961-e915-5b52-a517-30716352cb8e""","""2025-09-30 17:04:31""",
"""126f9c7d-5f8c-513f-9ec9-2ea7f70cde5a""","""2025-09-30 17:04:17""",
"""43e862df-c416-5ba9-a206-ba225543536e""",carÃ¡ter vocÃª nasce com ele para o bem ou p...,carter voc nasce bem mal
"""7ef15e91-8b8c-50ff-a64d-013af21eebfb""","""2025-09-30 17:03:18""",


## 5. Construção, Treinamento e Avaliação do Modelo

Esta é a etapa central do projeto. Com os dados limpos e pré-processados no DataFrame `df_final`, vamos:

1.  **Criar a Variável Alvo (`y`):** Como nosso dataset não possui uma coluna de sentimento, usaremos a coluna `score` (a pontuação do comentário) como um substituto (*proxy*). Comentários com score alto (>1) serão considerados "positivos", e os demais, "negativos/neutros".
2.  **Dividir os Dados:** Separar o dataset em um conjunto de **treino** (para o modelo aprender) e um conjunto de **teste** (para avaliarmos o quão bem ele aprendeu).
3.  **Construir um `Pipeline`:** Criar uma esteira de produção que primeiro transforma o texto limpo em vetores numéricos (`TfidfVectorizer`) e depois os usa para treinar um modelo de classificação (`LogisticRegression`).
4.  **Treinar e Avaliar:** Executar o treinamento e verificar a performance do modelo com métricas como a acurácia.

In [23]:
## 5. Construção, Treinamento e Avaliação do Modelo

print("\n--- Iniciando a Construção e Treinamento do Modelo ---")

# 1. Definir X (features) e y (target)
# Usamos a coluna 'texto_limpo' (do Bloco 4) e a coluna 'sentimento' (do Bloco 3)
print("[Passo 1/4] Definindo variáveis X e y...")
X = df['texto_limpo']
y = df['sentimento']

# 2. Dividir em treino e teste
# O stratify=y garante que a divisão mantenha a proporção de sentimentos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Total de amostras de treino: {len(X_train):,} amostras. Total de teste: {len(X_test):,} amostras.")

# 3. Construir o Pipeline do modelo
print("\n[Passos 3/4] Construindo o pipeline do modelo...")
# O Pipeline aplica primeiro o TF-IDF (vetorização) e depois a Regressão Logística (classificação)
modelo_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# 4. Treinar o modelo
print("\n[Passos 4/4] Iniciando o treinamento do modelo (isso leva alguns minutos)...")
modelo_pipeline.fit(X_train, y_train)
print("✅ Treinamento concluído!")

# 5. Fazer previsões no conjunto de teste e avaliar o modelo
y_pred = modelo_pipeline.predict(X_test)
acuracia = accuracy_score(y_test, y_pred)

print("\n--- PERFORMANCE DO MODELO ---")
print(f"Acurácia: {acuracia:.4f}")

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

--- Iniciando a Construção e Treinamento do Modelo ---

[Passo 1/5] Criando a variável de sentimento a partir do 'score'...
Distribuição das nossas classes (baseado no score):
score
positivo           124093
negativo_neutro     65160
Name: count, dtype: int64

[Passo 2/5] Dividindo o dataset em treino e teste...
151,402 amostras de treino, 37,851 amostras de teste.

[Passo 3/5] Construindo o pipeline do modelo...

[Passo 4/5] Iniciando o treinamento do modelo (pode levar um momento)...
✅ Treinamento concluído!

[Passo 5/5] Fazendo previsões e avaliando a performance...

--- PERFORMANCE DO MODELO ---
Acurácia: 0.6526

Relatório de Classificação:
                 precision    recall  f1-score   support

negativo_neutro       0.45      0.04      0.08     13032
       positivo       0.66      0.97      0.79     24819

       accuracy                           0.65     37851
      macro avg       0.56      0.51      0.43     37851
   weighted avg       0.59      0.65      0.54     37851



## 6. Salvando o Pipeline Final

Com o modelo treinado e avaliado, a etapa final é salvar o objeto `modelo_pipeline` em um arquivo. Isso nos permite reutilizar o modelo treinado no futuro (por exemplo, em nosso dashboard com Streamlit) sem a necessidade de retreiná-lo toda vez.

Usaremos a biblioteca `joblib` para salvar o modelo e, em seguida, a `google.colab.files` para baixá-lo para nosso computador.

In [24]:
import joblib
from google.colab import files

# Define o nome do arquivo que vai guardar nosso modelo
nome_arquivo_modelo = 'modelo_sentimento.joblib'

print(f"--- Salvando o modelo treinado em: {nome_arquivo_modelo} ---")

# Salva o objeto completo do pipeline no armazenamento local da sessão do Colab
joblib.dump(modelo_pipeline, nome_arquivo_modelo)

print("\n✅ Modelo salvo com sucesso no ambiente do Colab!")
print("Iniciando o download do modelo para o seu computador...")

# Gera o link para download do arquivo do modelo
files.download(nome_arquivo_modelo)


--- SALVAMENTO DO PIPELINE ---
✅ Pipeline do modelo salvo com sucesso em: /content/drive/MyDrive/Projeto_CampusParty_Sentimentos/saved_models/modelo_sentimento.joblib


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>